In [6]:
# import module
import os
import sys

# import dowhy
import econml
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import shap
import statsmodels.nonparametric.smoothers_lowess as sl
from econml.grf import CausalForest, CausalIVForest, RegressionForest
from econml.iv.dml import DMLIV, NonParamDMLIV, OrthoIV
from econml.iv.dr import DRIV, ForestDRIV, LinearDRIV, SparseLinearDRIV
from econml.sklearn_extensions.linear_model import WeightedLassoCV
from matplotlib import cm
from matplotlib.colors import Normalize
from scipy import special
from scipy.interpolate import interp1d, interpn
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import (LinearRegression, LogisticRegression,
                                  LogisticRegressionCV)
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from statsmodels.stats.multitest import multipletests

In [7]:
def warn(*args, **kwargs):
    pass
import warnings

warnings.warn = warn

In [8]:
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

In [10]:
# load dataset and sample
X_set1 = pd.read_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/04_pheno_covar_data/traits/ukbb.covariate.traits.set1.gz", sep = "\t")
X_set2 = pd.read_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/04_pheno_covar_data/traits/ukbb.covariate.traits.set2.gz", sep = "\t")
X_set3 = pd.read_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/04_pheno_covar_data/CRP/CAD/ukbb.covariate.CRP.set3.gz", sep = "\t")

Z_set1 = pd.read_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/06_PRS_calculation/CRP/CAD/set1/1e-08/CRP_prs.best", sep = " ") # score_constd
Z_set2 = pd.read_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/06_PRS_calculation/CRP/CAD/set2/1e-08/CRP_prs.best", sep = " ") # score_constd
Z_set3 = pd.read_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/06_PRS_calculation/CRP/CAD/set3/1e-08/CRP_prs.best", sep = " ") # score_constd

W = pd.read_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/04_pheno_covar_data/CRP/CAD/ukbb.phenotype.CRP.mgdL", sep = "\t")

Y_date1 = pd.read_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/03_outcome_data/CAD_outcome_include_comorbid_date1_james2022.gz", sep = "\t")
Y_date2 = pd.read_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/03_outcome_data/CAD_outcome_include_comorbid_date2_james2022.gz", sep = "\t")
Y_date3 = pd.read_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/03_outcome_data/CAD_outcome_include_comorbid_date3_james2022.gz", sep = "\t")

# harmonize the data
selected_id_set3 = set.intersection(set(X_set3['IID']), set(Z_set3['IID']), set(W['IID']), set(Y_date3['IID']) )
selected_id_set3 = list(selected_id_set3)
selected_id_set3_arr = np.array(selected_id_set3)
selected_id_set3_arr.sort()

selected_id_set1b = set.intersection(set(X_set1['IID']), set(Z_set1['IID']), set(W['IID']), set(Y_date3['IID']) )
selected_id_set1b = list(selected_id_set1b)
selected_id_set1b_arr = np.array(selected_id_set1b)
selected_id_set1b_arr.sort()

# model 1b
X_model1b = X_set1.loc[X_set1['IID'].isin(selected_id_set1b)].reset_index(drop = True)
Z_model1b = Z_set1.loc[Z_set1['IID'].isin(selected_id_set1b)].reset_index(drop = True)
W_model1b = W.loc[W['IID'].isin(selected_id_set1b)].reset_index(drop = True)
Y_model1b = Y_date3.loc[Y_date3['IID'].isin(selected_id_set1b)].reset_index(drop = True)

# model 3
X_model3 = X_set3.loc[X_set3['IID'].isin(selected_id_set3)].reset_index(drop = True)
Z_model3 = Z_set3.loc[Z_set3['IID'].isin(selected_id_set3)].reset_index(drop = True)
W_model3 = W.loc[W['IID'].isin(selected_id_set3)].reset_index(drop = True)
Y_model3 = Y_date3.loc[Y_date3['IID'].isin(selected_id_set3)].reset_index(drop = True)
X_model3 = pd.concat([X_model3, Y_model3.loc[:, ["htn", "t2dm", "heart_failure", "hemorrhage_stroke", "ischemic_stroke"]]], axis=1)
Y_model3 = Y_model3.loc[:, ["IID", "CAD"]]

# generate mat file for two models
# model 1b
W_model1b_mat = W_model1b.loc[:, ["30710-0.0"]]["30710-0.0"]
X_model1b_mat = X_model1b.iloc[:, 2:]
Y_model1b_mat = Y_model1b.loc[:, ["CAD"]]["CAD"]
Z_model1b_mat = Z_model1b.iloc[:, 3]

# model 3
W_model3_mat = W_model3.loc[:, ["30710-0.0"]]["30710-0.0"]
X_model3_mat = X_model3.iloc[:, 2:]
Y_model3_mat = Y_model3.loc[:, ["CAD"]]["CAD"]
Z_model3_mat = Z_model3.iloc[:, 3]

# correct the data type
X_model1b_mat = X_model1b_mat.astype({
    "22001-0.0": 'int64', "21022-0.0": 'float64', "22000-0.0": 'float64',
    "22009-0.1": 'float64', "22009-0.2": 'float64', "22009-0.3": 'float64', "22009-0.4": 'float64', "22009-0.5": 'float64',
    "22009-0.6": 'float64', "22009-0.7": 'float64', "22009-0.8": 'float64', "22009-0.9": 'float64', "22009-0.10": 'float64',
})

X_model3_mat = X_model3_mat.astype({
    "22001-0.0": 'int64', "21022-0.0": 'float64', 
    "4079-0.0": 'float64', "4080-0.0": 'float64', "189-0.0": 'float64', 
    "22009-0.1": 'float64', "22009-0.2": 'float64', "22009-0.3": 'float64', "22009-0.4": 'float64', "22009-0.5": 'float64',
    "22009-0.6": 'float64', "22009-0.7": 'float64', "22009-0.8": 'float64', "22009-0.9": 'float64', "22009-0.10": 'float64', "22000-0.0": 'float64',
    "21001-0.0": 'float64', "23099-0.0": 'float64', "21002-0.0": 'float64', # "whr": 'float64', 
    "Blood_pressure_medication": 'int64', "Cholesterol_lowering_medication": 'int64', "Insulin": 'int64', # "No_medication": 'int64', 
    "Non_alcohol_drinker": 'int64' , "Previous_alcohol_drinker": 'int64', "Current_alcohol_drinker": 'int64',
    "Non_smoker": 'int64' , "Previous_smoker": 'int64', "Current_smoker": 'int64',
    "30630-0.0": 'float64', "30760-0.0": 'float64', "30870-0.0": 'float64', # lipid-related covariates
    "30700-0.0": 'float64',  "30720-0.0": 'float64', "30730-0.0": 'float64', "30680-0.0": 'float64', 
    "30740-0.0": 'float64', "30750-0.0": 'float64', "30650-0.0": 'float64', "30660-0.0": 'float64', 
    "30670-0.0": 'float64', "30770-0.0": 'float64', "30810-0.0": 'float64', "30830-0.0": 'float64', "30850-0.0": 'float64', 
    "30880-0.0": 'float64', "30890-0.0": 'float64', "30840-0.0": 'float64', "30860-0.0": 'float64', 
    "t2dm": 'int64', "htn": 'int64', "heart_failure": 'int64', "hemorrhage_stroke": 'int64', "ischemic_stroke": 'int64'
})

# Rename the data covariates
X_model1b_mat.rename({
    "22001-0.0": 'Gender', "21022-0.0": 'Age',
    "22009-0.1": 'PC1', "22009-0.2": 'PC2', "22009-0.3": 'PC3', "22009-0.4": 'PC4', "22009-0.5": 'PC5',
    "22009-0.6": 'PC6', "22009-0.7": 'PC7', "22009-0.8": 'PC8', "22009-0.9": 'PC9', "22009-0.10": 'PC10', "22000-0.0": 'Genotype batch',
}, inplace=True)

X_model3_mat.rename({
    "22001-0.0": 'Gender', "21022-0.0": 'Age', 
    "4079-0.0": 'diastolic blood pressure', "4080-0.0": 'systolic blood pressure', "189-0.0": 'Townsend deprivation index', 
    "22009-0.1": 'PC1', "22009-0.2": 'PC2', "22009-0.3": 'PC3', "22009-0.4": 'PC4', "22009-0.5": 'PC5',
    "22009-0.6": 'PC6', "22009-0.7": 'PC7', "22009-0.8": 'PC8', "22009-0.9": 'PC9', "22009-0.10": 'PC10', "22000-0.0": 'Genotype batch',
    "21001-0.0": 'BMI', "23099-0.0": 'Body Fat Percentage', "21002-0.0": 'Weight', # "whr": 'Waist-hip-ratio', 
    "Blood_pressure_medication": 'Blood pressure medication', "Cholesterol_lowering_medication": 'Cholesterol lowering medication', "Insulin": 'Insulin', # "No_medication": 'No medication', 
    "Non_alcohol_drinker": 'Non-alcohol drinker' , "Previous_alcohol_drinker": 'Previous alcohol drinker', "Current_alcohol_drinker": 'Current alcohol drinker',
    "Non_smoker": 'Non-smoker' , "Previous_smoker": 'Previous smoker', "Current_smoker": 'Current smoker',
    "30630-0.0": 'Apolipoprotein A', "30760-0.0": 'HDL-C', "30870-0.0": 'Triglycerides', # lipid-related covariates
    "30700-0.0": 'Creatinine', "30720-0.0": 'Cystatin C', "30730-0.0": 'Gamma glutamyltransferase', "30680-0.0": 'Calcium', 
    "30740-0.0": 'Glucose', "30750-0.0": 'HbA1c', "30650-0.0": 'Aspartate aminotransferase', "30660-0.0": 'Direct bilirubin', 
    "30670-0.0": 'Urea', "30770-0.0": 'IGF-1', "30810-0.0": '30810-0.0', "30830-0.0": 'SHBG', "30850-0.0": 'Testosterone', 
    "30880-0.0": 'Urate', "30890-0.0": 'Vitamin D', "30840-0.0": 'Total bilirubin', "30860-0.0": 'Total protein', 
    "t2dm": 'Type 2 diabetes history', "htn": 'Hypertension history', "heart_failure": 'Heart failure history', "hemorrhage_stroke": 'Hemorrhage Stroke history', "ischemic_stroke": 'Ischemic Stroke history'
}, inplace=True)

In [11]:
Z_model3

,FID,IID,In_Regression,PRS
0,1000048,1000048,Yes,0.086921
1,1000055,1000055,Yes,-0.109392
2,1000067,1000067,Yes,-0.719151
3,1000072,1000072,Yes,0.329246
4,1000099,1000099,Yes,0.310084
...,...,...,...,...
276049,6026113,6026113,Yes,-0.357190
276050,6026124,6026124,Yes,1.312557
276051,6026136,6026136,Yes,2.220868
276052,6026155,6026155,Yes,0.886672


In [12]:
X_model3

,FID,IID,22001-0.0,21022-0.0,22000-0.0,4079-0.0,4080-0.0,189-0.0,Non_smoker,Previous_smoker,...,30860-0.0,30850-0.0,30870-0.0,30880-0.0,30890-0.0,htn,t2dm,heart_failure,hemorrhage_stroke,ischemic_stroke
0,1000048,1000048,0,47,-1,63,106,-2.33710,1,0,...,65.83,0.602,0.780,244.3,39.1,0,0,0,0,0
1,1000055,1000055,1,68,41,101,167,1.91830,0,1,...,76.17,12.016,3.138,308.8,47.4,1,1,0,0,0
2,1000067,1000067,0,53,33,91,156,-3.29994,0,1,...,72.77,0.851,0.860,205.6,80.5,0,0,0,0,0
3,1000072,1000072,0,58,-5,73,128,-1.44614,0,1,...,63.70,0.539,1.835,299.1,25.8,0,0,0,0,0
4,1000099,1000099,1,55,62,82,144,-0.12643,0,1,...,66.99,8.248,3.783,364.4,58.2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276049,6026113,6026113,1,66,1,70,127,1.08555,0,0,...,77.34,14.558,1.240,330.3,87.5,1,0,0,0,0
276050,6026124,6026124,1,59,90,95,178,-2.58202,0,1,...,78.50,7.904,1.308,395.5,57.4,1,0,0,0,0
276051,6026136,6026136,1,63,29,81,153,-4.52068,0,1,...,70.21,7.945,1.981,363.5,27.3,0,1,0,0,0
276052,6026155,6026155,0,49,48,84,129,-0.32123,0,1,...,80.99,0.780,1.865,518.5,30.1,1,0,0,0,0


In [13]:
W_model3

,FID,IID,30710-0.0
0,1000048,1000048,0.035
1,1000055,1000055,0.235
2,1000067,1000067,0.077
3,1000072,1000072,0.331
4,1000099,1000099,0.078
...,...,...,...
276049,6026113,6026113,0.652
276050,6026124,6026124,0.261
276051,6026136,6026136,0.370
276052,6026155,6026155,4.717


In [14]:
W_model1b_mat_binary = np.where(W_model1b_mat >= 2, 1, 0)
W_model1b_mat_binary = pd.DataFrame({"30710-0.0": W_model1b_mat_binary}).iloc[:, 0]
W_model3_mat_binary = np.where(W_model3_mat >= 2, 1, 0)
W_model3_mat_binary = pd.DataFrame({"30710-0.0": W_model3_mat_binary}).iloc[:, 0]

In [9]:
# Z_model1_mat_binary = np.where(Z_model1_mat <= 0, 1, 0)
# Z_model1_mat_binary = pd.DataFrame({"PRS": Z_model1_mat_binary}).iloc[:, 0]
# Z_model1b_mat_binary = np.where(Z_model1b_mat <= 0, 1, 0)
# Z_model1b_mat_binary = pd.DataFrame({"PRS": Z_model1b_mat_binary}).iloc[:, 0]
# Z_model2_mat_binary = np.where(Z_model2_mat <= 0, 1, 0)
# Z_model2_mat_binary = pd.DataFrame({"PRS": Z_model2_mat_binary}).iloc[:, 0]
# Z_model2b_mat_binary = np.where(Z_model2b_mat <= 0, 1, 0)
# Z_model2b_mat_binary = pd.DataFrame({"PRS": Z_model2b_mat_binary}).iloc[:, 0]
# Z_model3_mat_binary = np.where(Z_model3_mat <= 0, 1, 0)
# Z_model3_mat_binary = pd.DataFrame({"PRS": Z_model3_mat_binary}).iloc[:, 0]

#### DRIV estimator

#### Model 1

In [ ]:
# # split the dataset into training and testing
# train, test = train_test_split(X_model1_mat, test_size=0.5, stratify=Y_model1_mat, random_state=309)
# train_set = train.index.to_list()
# test_set = test.index.to_list()
# train_set.sort()
# test_set.sort()

# # write the training set and testing set to file, which will be used in R analysis.
# train_set_pd = pd.DataFrame({"Training_index": [x+1 for x in train_set], "Training_id": selected_id_set1_arr[train_set]})
# test_set_pd = pd.DataFrame({"Testing_index": [x+1 for x in test_set], "Testing_id": selected_id_set1_arr[test_set]})

In [ ]:
# # Continuous W (Training/Testing Set)
# est_driv_train_continuousW = ForestDRIV(projection=False, discrete_treatment=False, discrete_instrument=False, \
#                                         n_estimators=5000, min_samples_leaf=50, max_samples=0.02, 
#                                         random_state=309, cov_clip = 1, n_jobs=15)
# est_driv_train_continuousW.fit(Y_model1_mat[train_set], W_model1_mat[train_set], Z=Z_model1_mat[train_set], X=X_model1_mat.loc[train_set])
# point_driv_test_continuousW = est_driv_train_continuousW.effect(X_model1_mat.loc[test_set])
# point_driv_train_continuousW = est_driv_train_continuousW.effect(X_model1_mat.loc[train_set]) 
# print(pd.DataFrame({"dat": point_driv_train_continuousW}).describe())
# print(pd.DataFrame({"dat": point_driv_test_continuousW}).describe())

In [ ]:
# # Continuous W (Full Set)
# est_driv_continuousW_model1 = ForestDRIV(projection=False, discrete_treatment=False, discrete_instrument=False, \
#                                          n_estimators=5000, min_samples_leaf=100, max_samples=0.02, 
#                                          random_state=309, cov_clip = 0.01, n_jobs=15) 
# est_driv_continuousW_model1.fit(Y_model1_mat, W_model1_mat, Z=Z_model1_mat, X=X_model1_mat, cache_values=True)
# point_driv_continuousW_model1 = est_driv_continuousW_model1.effect(X_model1_mat)
# print(pd.DataFrame({"dat": point_driv_continuousW_model1}).describe())

In [15]:
# Continuous W (Full Set)
est_driv_continuousW_model1b = ForestDRIV(projection=False, discrete_treatment=False, discrete_instrument=False, \
                                         n_estimators=5000, min_samples_leaf=100, max_samples=0.02, 
                                         random_state=309, cov_clip = 0.01, n_jobs=35) 
est_driv_continuousW_model1b.fit(Y_model1b_mat, W_model1b_mat, Z=Z_model1b_mat, X=X_model1b_mat, cache_values=True)
point_driv_continuousW_model1b = est_driv_continuousW_model1b.effect(X_model1b_mat)
print(pd.DataFrame({"dat": point_driv_continuousW_model1b}).describe())

                 dat
count  276054.000000
mean        0.004696
std         0.008103
min        -0.027389
25%        -0.000837
50%         0.004614
75%         0.010115
max         0.042896


In [ ]:
# # Continuous W (Full Set)
# est_driv_continuousW_model2 = ForestDRIV(projection=False, discrete_treatment=False, discrete_instrument=False, \
#                                          n_estimators=5000, min_samples_leaf=100, max_samples=0.02, 
#                                          random_state=309, cov_clip = 10, n_jobs=15) 
# est_driv_continuousW_model2.fit(Y_model2_mat, W_model2_mat, Z=Z_model2_mat, X=X_model2_mat, cache_values=True)
# point_driv_continuousW_model2 = est_driv_continuousW_model2.effect(X_model2_mat)
# print(pd.DataFrame({"dat": point_driv_continuousW_model2}).describe())

In [ ]:
# # Continuous W (Full Set)
# est_driv_continuousW_model2b = ForestDRIV(projection=False, discrete_treatment=False, discrete_instrument=False, \
#                                          n_estimators=5000, min_samples_leaf=100, max_samples=0.02, 
#                                          random_state=309, cov_clip = 10, n_jobs=15) 
# est_driv_continuousW_model2b.fit(Y_model2b_mat, W_model2b_mat, Z=Z_model2b_mat, X=X_model2b_mat, cache_values=True)
# point_driv_continuousW_model2b = est_driv_continuousW_model2b.effect(X_model2b_mat)
# print(pd.DataFrame({"dat": point_driv_continuousW_model2b}).describe())

In [5]:
# Continuous W (Full Set)
est_driv_continuousW_model3 = ForestDRIV(projection=False, discrete_treatment=False, discrete_instrument=False, \
                                  n_estimators=5000, min_samples_leaf=100, max_samples=0.02, 
                                  random_state=309, cov_clip = 5, n_jobs=15) 
est_driv_continuousW_model3.fit(Y_model3_mat, W_model3_mat, Z=Z_model3_mat, X=X_model3_mat, cache_values=True)
point_driv_continuousW_model3 = est_driv_continuousW_model3.effect(X_model3_mat)
print(pd.DataFrame({"dat": point_driv_continuousW_model3}).describe())

                 dat
count  276054.000000
mean        0.000349
std         0.000076
min         0.000029
25%         0.000298
50%         0.000347
75%         0.000399
max         0.000673


In [ ]:
# # Binary W (Full Set)
# est_driv_binaryW_model1 = ForestDRIV(projection=False, discrete_treatment=False, discrete_instrument=False, 
#                                      n_estimators=5000, min_samples_leaf=100, max_samples=0.02, 
#                                      random_state=309, cov_clip = 0.1, n_jobs=15) 
# est_driv_binaryW_model1.fit(Y_model1_mat, W_model1_mat_binary, Z=Z_model1_mat, X=X_model1_mat, cache_values=True)
# point_driv_binaryW_model1 = est_driv_binaryW_model1.effect(X_model1_mat)
# print(pd.DataFrame({"dat": point_driv_binaryW_model1}).describe())

In [ ]:
# # Binary W (Full Set)
# est_driv_binaryW_model2 = ForestDRIV(projection=False, discrete_treatment=False, discrete_instrument=False,
#                                      n_estimators=5000, min_samples_leaf=100, max_samples=0.02, 
#                                      random_state=309, cov_clip = 0.1, n_jobs=15) 
# est_driv_binaryW_model2.fit(Y_model2_mat, W_model2_mat_binary, Z=Z_model2_mat, X=X_model2_mat, cache_values=True)
# point_driv_binaryW_model2 = est_driv_binaryW_model2.effect(X_model2_mat)
# print(pd.DataFrame({"dat": point_driv_binaryW_model2}).describe())

In [6]:
# Binary W (Full Set)
est_driv_binaryW_model3 = ForestDRIV(projection=False, discrete_treatment=False, discrete_instrument=False, \
                                         n_estimators=5000, min_samples_leaf=100, max_samples=0.02, 
                                         random_state=309, cov_clip = 0.1, n_jobs=15) 
est_driv_binaryW_model3.fit(Y_model3_mat, W_model3_mat_binary, Z=Z_model3_mat, X=X_model3_mat, cache_values=True)
point_driv_binaryW_model3 = est_driv_binaryW_model3.effect(X_model3_mat)
print(pd.DataFrame({"dat": point_driv_binaryW_model3}).describe())

                 dat
count  276054.000000
mean       -0.026608
std         0.005103
min        -0.049984
25%        -0.030055
50%        -0.026231
75%        -0.022938
max        -0.008567


In [ ]:
point_driv_lb_continuousW_model1, point_driv_ub_continuousW_model1 = est_driv_continuousW_model1.effect_interval(X_model1_mat, alpha=0.05) # type: ignore
z_value_driv_continuousW_model1 = point_driv_continuousW_model1/((point_driv_ub_continuousW_model1-point_driv_lb_continuousW_model1)/(2*1.96))
p_value_driv_continuousW_model1 = scipy.stats.norm.sf(abs(z_value_driv_continuousW_model1)) # * 2
p_value_BH_driv_continuousW_model1 = multipletests(pvals = p_value_driv_continuousW_model1, method = "fdr_bh", alpha=0.1)

full_results_continuousW_model1 = pd.DataFrame({"IID": selected_id_set1_arr, "point": point_driv_continuousW_model1, "upper_bound": point_driv_ub_continuousW_model1, \
                                "lower_bound": point_driv_lb_continuousW_model1, "z-value": z_value_driv_continuousW_model1, \
                                "p_value": p_value_driv_continuousW_model1, "p_value_corrected": p_value_BH_driv_continuousW_model1[1]})

# model 2
point_driv_lb_continuousW_model2, point_driv_ub_continuousW_model2 = est_driv_continuousW_model2.effect_interval(X_model2_mat, alpha=0.05) # type: ignore
z_value_driv_continuousW_model2 = point_driv_continuousW_model2/((point_driv_ub_continuousW_model2-point_driv_lb_continuousW_model2)/(2*1.96))
p_value_driv_continuousW_model2 = scipy.stats.norm.sf(abs(z_value_driv_continuousW_model2)) # * 2
p_value_BH_driv_continuousW_model2 = multipletests(pvals = p_value_driv_continuousW_model2, method = "fdr_bh", alpha=0.1)

full_results_continuousW_model2 = pd.DataFrame({"IID": selected_id_set2_arr, "point": point_driv_continuousW_model2, "upper_bound": point_driv_ub_continuousW_model2, \
                                "lower_bound": point_driv_lb_continuousW_model2, "z-value": z_value_driv_continuousW_model2, \
                                "p_value": p_value_driv_continuousW_model2, "p_value_corrected": p_value_BH_driv_continuousW_model2[1]})

# model 3
point_driv_lb_continuousW_model3, point_driv_ub_continuousW_model3 = est_driv_continuousW_model3.effect_interval(X_model3_mat, alpha=0.05) # type: ignore
z_value_driv_continuousW_model3 = point_driv_continuousW_model3/((point_driv_ub_continuousW_model3-point_driv_lb_continuousW_model3)/(2*1.96))
p_value_driv_continuousW_model3 = scipy.stats.norm.sf(abs(z_value_driv_continuousW_model3)) # * 2
p_value_BH_driv_continuousW_model3 = multipletests(pvals = p_value_driv_continuousW_model3, method = "fdr_bh", alpha=0.1)

full_results_continuousW_model3 = pd.DataFrame({"IID": selected_id_set3_arr, "point": point_driv_continuousW_model3, "upper_bound": point_driv_ub_continuousW_model3, \
                                "lower_bound": point_driv_lb_continuousW_model3, "z-value": z_value_driv_continuousW_model3, \
                                "p_value": p_value_driv_continuousW_model3, "p_value_corrected": p_value_BH_driv_continuousW_model3[1]})


In [ ]:
full_results_continuousW_model3.describe()

In [ ]:
point_driv_lb_binaryW_model1, point_driv_ub_binaryW_model1 = est_driv_binaryW_model1.effect_interval(X_model1_mat, alpha=0.05) # type: ignore
z_value_driv_binaryW_model1 = point_driv_binaryW_model1/((point_driv_ub_binaryW_model1-point_driv_lb_binaryW_model1)/(2*1.96))
p_value_driv_binaryW_model1 = scipy.stats.norm.sf(abs(z_value_driv_binaryW_model1)) # * 2
p_value_BH_driv_binaryW_model1 = multipletests(pvals = p_value_driv_binaryW_model1, method = "fdr_bh", alpha=0.1)

full_results_binaryW_model1 = pd.DataFrame({"IID": selected_id_set1_arr, "point": point_driv_binaryW_model1, "upper_bound": point_driv_ub_binaryW_model1, \
                                "lower_bound": point_driv_lb_binaryW_model1, "z-value": z_value_driv_binaryW_model1, \
                                "p_value": p_value_driv_binaryW_model1, "p_value_corrected": p_value_BH_driv_binaryW_model1[1]})

# model 2
point_driv_lb_binaryW_model2, point_driv_ub_binaryW_model2 = est_driv_binaryW_model2.effect_interval(X_model2_mat, alpha=0.05) # type: ignore
z_value_driv_binaryW_model2 = point_driv_binaryW_model2/((point_driv_ub_binaryW_model2-point_driv_lb_binaryW_model2)/(2*1.96))
p_value_driv_binaryW_model2 = scipy.stats.norm.sf(abs(z_value_driv_binaryW_model2)) # * 2
p_value_BH_driv_binaryW_model2 = multipletests(pvals = p_value_driv_binaryW_model2, method = "fdr_bh", alpha=0.1)

full_results_binaryW_model2 = pd.DataFrame({"IID": selected_id_set2_arr, "point": point_driv_binaryW_model2, "upper_bound": point_driv_ub_binaryW_model2, \
                                "lower_bound": point_driv_lb_binaryW_model2, "z-value": z_value_driv_binaryW_model2, \
                                "p_value": p_value_driv_binaryW_model2, "p_value_corrected": p_value_BH_driv_binaryW_model2[1]})

# model 3
point_driv_lb_binaryW_model3, point_driv_ub_binaryW_model3 = est_driv_binaryW_model3.effect_interval(X_model3_mat, alpha=0.05) # type: ignore
z_value_driv_binaryW_model3 = point_driv_binaryW_model3/((point_driv_ub_binaryW_model3-point_driv_lb_binaryW_model3)/(2*1.96))
p_value_driv_binaryW_model3 = scipy.stats.norm.sf(abs(z_value_driv_binaryW_model3)) # * 2
p_value_BH_driv_binaryW_model3 = multipletests(pvals = p_value_driv_binaryW_model3, method = "fdr_bh", alpha=0.1)

full_results_binaryW_model3 = pd.DataFrame({"IID": selected_id_set3_arr, "point": point_driv_binaryW_model3, "upper_bound": point_driv_ub_binaryW_model3, \
                                "lower_bound": point_driv_lb_binaryW_model3, "z-value": z_value_driv_binaryW_model3, \
                                "p_value": p_value_driv_binaryW_model3, "p_value_corrected": p_value_BH_driv_binaryW_model3[1]})


In [ ]:
full_results_binaryW_model1.describe()

In [7]:
shap_values_driv_binaryW_model3 = est_driv_binaryW_model3.shap_values(X_model3_mat.iloc[1:100, :])


 96%|=================== | 95/99 [00:23<00:00]       

In [9]:
pd.DataFrame(shap_values_driv_binaryW_model3['CAD']['30780-0.0'].values)

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,-7.078144e-06,-0.000550,-0.000041,-0.000254,0.001158,-0.000368,0.000013,-5.244219e-05,-0.000006,0.0,...,-0.000516,0.000581,-0.000900,0.000639,0.000011,0.000010,-0.000635,-0.000045,0.0,0.0
1,9.748803e-07,-0.000282,0.000095,-0.000461,-0.000326,0.000047,0.000031,2.876065e-05,0.000029,0.0,...,0.000362,0.000480,-0.000235,0.000846,0.000101,0.000003,0.000078,-0.000032,0.0,0.0
2,-4.780498e-07,-0.000061,0.000882,0.000313,0.000498,-0.000247,-0.000044,-3.424246e-05,0.000015,0.0,...,0.000676,-0.000681,0.000327,0.000694,-0.000040,-0.000219,0.000076,0.000012,0.0,0.0
3,2.638204e-07,0.000077,-0.000530,0.000090,-0.000623,0.000160,0.000018,-4.898037e-05,0.000025,0.0,...,-0.000150,-0.000969,-0.002209,-0.000709,-0.000243,-0.000070,0.000067,-0.000062,0.0,0.0
4,3.543072e-06,-0.000363,-0.000734,0.000043,0.000324,0.000345,-0.000031,-3.268312e-07,0.000023,0.0,...,0.000392,0.000579,-0.000437,0.000376,-0.000405,-0.000161,0.000078,-0.000032,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,-5.189410e-06,0.000370,0.000334,-0.000175,0.002413,0.000464,0.000072,6.953259e-05,0.000002,0.0,...,-0.001416,-0.000975,-0.000019,-0.000029,0.000040,0.000235,-0.000618,0.000801,0.0,0.0
95,1.733803e-06,0.001227,-0.000228,0.000081,0.000177,0.000343,0.000059,-1.907442e-05,0.000023,0.0,...,0.000677,-0.000921,-0.000279,0.000483,-0.000210,-0.000197,0.000065,0.001294,0.0,0.0
96,-1.027698e-06,0.000260,-0.000225,-0.000207,0.000006,-0.001366,0.000015,-1.674840e-05,0.000005,0.0,...,-0.001270,-0.000837,0.000385,-0.001078,-0.000475,-0.000139,0.000093,-0.000094,0.0,0.0
97,-3.517214e-06,-0.000075,-0.000810,0.000359,0.000341,0.000141,0.000044,-6.489420e-05,0.000021,0.0,...,-0.000713,0.000341,-0.000383,-0.000744,-0.000433,0.000114,0.000061,-0.000083,0.0,0.0


In [10]:
save_object(shap_values_driv_binaryW_model3, "/home/yujia/Project/2023-07-20-individual_MR/res/02_ITE_analysis/01_table/LDL/CAD/03_variable_importance/driv_binaryW_shap_model3.pkl")

In [12]:
def pickle_loader(filename):
    """ Deserialize a file of pickled objects. """
    with open(filename, "rb") as f:
        while True:
            try:
                yield pickle.load(f)
            except EOFError:
                break

In [13]:
aaa = pickle_loader("/home/yujia/Project/2023-07-20-individual_MR/res/02_ITE_analysis/01_table/LDL/CAD/03_variable_importance/driv_binaryW_shap_model3.pkl")

In [18]:
with open ("/home/yujia/Project/2023-07-20-individual_MR/res/02_ITE_analysis/01_table/LDL/CAD/03_variable_importance/driv_binaryW_shap_model3.pkl", 'rb') as f: #打开文件
    t3 = pickle.load(f)

In [23]:
t3["CAD"]

{'30780-0.0': .values =
 array([[-7.07814380e-06, -5.49878006e-04, -4.14469086e-05, ...,
         -4.52179193e-05,  0.00000000e+00,  0.00000000e+00],
        [ 9.74880308e-07, -2.81987546e-04,  9.45016304e-05, ...,
         -3.21802036e-05,  0.00000000e+00,  0.00000000e+00],
        [-4.78049831e-07, -6.07934448e-05,  8.82420563e-04, ...,
          1.15306453e-05,  0.00000000e+00,  0.00000000e+00],
        ...,
        [-1.02769804e-06,  2.60230154e-04, -2.24803440e-04, ...,
         -9.35933208e-05,  0.00000000e+00,  0.00000000e+00],
        [-3.51721370e-06, -7.52364643e-05, -8.09684763e-04, ...,
         -8.26829959e-05,  0.00000000e+00,  0.00000000e+00],
        [ 7.37915021e-06, -1.60318054e-04,  5.02162978e-04, ...,
         -5.71548245e-05,  0.00000000e+00,  0.00000000e+00]])
 
 .base_values =
 array([-0.02695073, -0.02695073, -0.02695073, -0.02695073, -0.02695073,
        -0.02695073, -0.02695073, -0.02695073, -0.02695073, -0.02695073,
        -0.02695073, -0.02695073, -0.02695